# Bulk Forecast

Geração de previsões para multiplas categorias

In [34]:
import pandas as pd
import numpy as np
import json

from fbprophet import Prophet

in_data = pd.read_csv('geral.csv', index_col=0)
categories = json.load(open('subcategorias.json'))
out_data = pd.DataFrame(np.zeros((len(categories.keys()), 11)), columns=['Mes', 'Ano', 'Categoria', 'Produto', 'Compra sugerida', 'Venda prevista', 'Venda anterior', 'Estoque atual', 'Estoque anterior', 'Valor venda', 'Valor anterior'])

In [103]:
def get_forecast_figure(train, periods):
    ph = Prophet(daily_seasonality=False)
    ph.fit(pd.DataFrame({'ds': train.index, 'y':train.values}))

    future = ph.make_future_dataframe(periods=periods, freq='D')
    forecast = ph.predict(future)
    
    forecast.tail()
    
    return np.round(forecast['yhat'].sum())

In [144]:
i = 0
for key in categories.keys():
    in_data[key] = in_data[categories[key]].sum(axis=1)
    row = pd.Series([3, 2021, 
                     key, key, 
                     np.random.randint(0, 99), # Compra sugerida
                     np.random.randint(0, 999), # Venda prevista
                     in_data.loc['2021-03-01':'2021-03-31', key].sum(), # Venda anterior
                     np.random.randint(0, 99), # Estoque atual
                     np.random.randint(0, 99), # Estoque anterior
                     np.random.randint(0, 999999), # Valor venda
                     np.random.randint(0, 999999)]) # Valor anterior
    out_data.iloc[i] = row
    i+= 1

In [145]:
out_data.to_csv('fatos.csv')